In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd

## Part Three

In [2]:
user = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
dbname = "topChef"
port = 3306
dburi = f"mysql+pymysql://{user}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [3]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()

def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    return result

In [4]:
tables = df_select("show tables;", engine)
tables

,Tables_in_topChef
0,challengedescriptions
1,challengewins
2,chefdetails
3,episodeinfo
4,judges
5,rewards


### 1. 

In [5]:
query_1 = """ SELECT * FROM chefdetails
              WHERE season NOT LIKE '%Masters%'
              AND season NOT LIKE '%Canada 6%'; """
chefdetails_clean = df_select(query_1, engine)

In [6]:
query_2 = """ SELECT * FROM judges
              WHERE season NOT LIKE '%Masters%'
              AND season NOT LIKE '%Canada 6%';"""
judges_clean = df_select(query_2, engine)

### 2. 

In [7]:
chefdetails_cols = ["season", "seasonNumber", "name", "placement", "gender"]
judges_cols = ["season", "seasonNumber", "episode", "challengeType", "guestJudge", "competedOnTC"]


In [16]:
chefdetails_clean = chefdetails_clean[chefdetails_cols]
judges_clean = judges_clean[judges_cols]
chefdetails_clean

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female
...,...,...,...,...,...
317,World All Stars,20,May Phattanant Thongthong,14,Female
318,World All Stars,20,Begona Rodrigo,12,Female
319,World All Stars,20,Gabriel Rodriguez,2,Male
320,World All Stars,20,Amar Santana,6,Male


In [17]:
judges_clean

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None
...,...,...,...,...,...,...
628,World All Stars,20,12,Quickfire,Sam Bompas,None
629,World All Stars,20,12,Elimination,Jeremy Chan,None
630,World All Stars,20,13,Quickfire,Greg Marchand,None
631,World All Stars,20,13,Elimination,Greg Marchand,None


### 3. 

In [15]:
season_df = chefdetails_clean[["season","seasonNumber"]].drop_duplicates().sort_values("seasonNumber", ignore_index = True)
season_df

,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


### Questions

#### a. 

In [18]:
chefdetails_clean["gender"].value_counts()

Male      170
Female    152
Name: gender, dtype: int64

Out of the remaining 20 seasons, $\textbf{170}$ of the contestants were mail, and $\textbf{152}$ were female. 

#### b. 

In [19]:
chefdetails_clean[chefdetails_clean["placement"] == 1]["gender"].value_counts()

Male      14
Female     6
Name: gender, dtype: int64

Among all winners, $\textbf{14}$ were male, and $\textbf{6}$ female.

#### c. 

In [20]:
chefdetails_clean[(chefdetails_clean["placement"] == 1) | (chefdetails_clean["placement"] == 2) | (chefdetails_clean["placement"] == 3)]["gender"].value_counts()


Male      36
Female    25
Name: gender, dtype: int64

Among all top 3 finalists, $\textbf{36}$ were male, and $\textbf{25}$ female.

#### d. 

To preface, I have zero background knowledge of this show and how it is judged. That being said, it seems fair to assume that if a female contestant places in the top 3 then she should have equal probabillity of finishing in any one of the top 3 positions. 

That is, $P(\text{placement} = i | \text{ finished in top 3}) = \dfrac{1}{3} \text{ for i} = 1,2,3$

We observed that the probability of winning given top 3 finisher is $\dfrac{6}{25}$. 

This probability, being less than $\dfrac{1}{3}$, indicates a non-uniform distribution of placement for top 3 female finishers. 

From part b above, we see that there were 20 winners (both male and female) for 20 seasons, however a quick analysis shows that for second place winners (both male and female) there were 29 contestants. This means some seasons had multiple second place finishers.

Some reading on wikipedia as well as a quick check of the following table, shows that often, many seasons had multiple "runner ups". This explains the lower than expected winning rate of top 3 female finishers. 

In [100]:
chefdetails_clean[chefdetails_clean["placement"]==2].sort_values("seasonNumber")

,season,seasonNumber,name,placement,gender
261,San Francisco,1,Tiffani Faison,2,Female
180,Los Angeles,2,Marcel Vigneron,2,Male
196,Miami,3,Dale Levitski,2,Male
195,Miami,3,Casey Thompson,2,Female
83,Chicago,4,Lisa Fernandes,2,Female
84,Chicago,4,Richard Blais,2,Male
230,New York,5,Stefan Richter,2,Male
229,New York,5,Carla Hall,2,Female
163,Las Vegas,6,Bryan Voltaggio,2,Male
164,Las Vegas,6,Kevin Gillespie,2,Male


### 4. 

In [22]:
query = """ SELECT DISTINCT chefdetails.name, chefdetails.season,
            CASE WHEN judges.guestJudge IS NOT NULL THEN 'yes' ELSE 'no' END
            AS 'guest judge?'
            
            FROM chefdetails
            LEFT JOIN judges ON chefdetails.name = judges.guestJudge
            WHERE chefdetails.placement = 1
                AND chefdetails.season NOT LIKE '%Masters%'
                AND chefdetails.season NOT LIKE '%Canada 6%'
                
            ORDER BY chefdetails.seasonNumber;"""

            
table_4 = df_select(query, engine)
table_4

,name,season,guest judge?
0,Harold Dieterle,San Francisco,yes
1,Ilan Hall,Los Angeles,yes
2,Hung Huynh,Miami,yes
3,Stephanie Izard,Chicago,yes
4,Hosea Rosenberg,New York,yes
5,Michael Voltaggio,Las Vegas,yes
6,Kevin Sbraga,D.C.,no
7,Richard Blais,All Stars: New York,yes
8,Paul Qui,Texas,no
9,Kristen Kish,Seattle,yes


In [23]:
query = """ SELECT DISTINCT chefdetails.name,
            CASE WHEN judges.guestJudge IS NOT NULL THEN 'yes' ELSE 'no' END
            AS gj
            
            FROM chefdetails
            LEFT JOIN judges ON chefdetails.name = judges.guestJudge
            WHERE chefdetails.placement != 1
                AND chefdetails.season NOT LIKE '%Masters%'
                AND chefdetails.season NOT LIKE '%Canada 6%'
            ORDER BY chefdetails.seasonNumber
            
            ;"""

            
df5 = df_select(query, engine)

In [24]:
df5 = df5[df5["gj"] == "yes"]["name"].reset_index(drop = True)

In [25]:
df5

0         Miguel Morales
1        Michael Midgley
2           Betty Fraser
3     Josie Smith-Malave
4          Richard Blais
5         Stefan Richter
6          Mike Isabella
7         Antonia Lofaso
8          Tiffany Derry
9             Dale Talde
10      Spike Mendelsohn
11      Stephen Asprinio
12            Edward Lee
13      Nyesha Arrington
14     Brooke Williamson
15          Nina Compton
16         Carlos Gaytan
17          Melissa King
18          Amar Santana
19        Kwame Onwuachi
20        Sheldon Simeon
21            Sam Talbot
22     Adrienne Cheatham
23          Carrie Baird
24       Bryan Voltaggio
25       Gregory Gourdet
26       Karen Akunowicz
27         Lee Anne Wong
28           Nini Nguyen
29          Dawn Burrell
30        Shota Nakajima
31          Nicole Gomes
Name: name, dtype: object